<table style="float:left; border:none">
   <tr style="border:none">
       <td style="border:none">
           <a href="https://bokeh.org/" target="_blank">
           <img
               src="assets/bokeh-transparent.png"
               style="width:50px"
           >
           </a>
       </td>
       <td style="border:none">
           <h1>Bokeh Tutorial</h1>
       </td>
   </tr>
</table>

<div style="float:right;"><a href="TOC.ipynb" target="_blank">Table of contents</a><br><h2>08 Plot tools and tooltips</h2></div>

In [ ]:
# load tutorial data
from tutorial_data import data

In [ ]:
# activate notebook output
from bokeh.io import output_notebook

output_notebook()

This chapter provides an overview of the tools that are available in the Bokeh toolbar.

![Example of Bokeh toolbar](assets/bokeh_toolbar.png)

The Bokeh toolbar is a collection of tools that are displayed together with a plot.
You have seen the toolbar in all previous examples. This chapter is about configuring
the toolbar and its tools.

### General toolbar configuration

By default, the Bokeh toolbar is displayed to the right of every plot. 

You can change this by setting the `toolbar_location` property of the figure:

In [ ]:
from bokeh.plotting import figure, show

# 🔁 Try different values for the toolbar_location parameter
p = figure(
    height=300,
    toolbar_location="below",  # can be "above", "below" "left", "right", or None
)

p.line(x=list(range(10)), y=list(range(10)))
show(p)

To completely disable the toolbar for a plot, set `toolbar_location` to `None`.

### Adding and removing tools

When you configure the toolbar with the ``figure()`` method, you can specify **which
tools to include**.

You can pass a sequence of tool names to the `tools` argument. The following list contains
some of the most commonly used tools for viewing data:

* `pan` - pan the plot
* `xpan` or `ypan` - pan the plot only in the x- or y-direction
* `wheel_zoom` - zoom the plot in and out using the mouse wheel
* `xwheel_zoom` or `ywheel_zoom` - zoom the plot only in the x- or y-direction
* `zoom_in` or `zoom_out` - zoom the plot in and out when clicking on this toolbar button
* `box_zoom` - zoom the plot in and out using a rectangular selection
* `save` - save the plot to a file
* `reset` - reset the plot to its original state
* `help` - display help about the available tools

Use the following code cell to try out different tools in the toolbar:

In [ ]:
from bokeh.plotting import figure

# set up a figure with a toolbar
p = figure(
    height=300,
    tools=("box_zoom", "wheel_zoom", "save", "reset"),  # 🔁 swap out tools from the list above
)

p.line(x=list(range(10)), y=list(range(10)))
show(p)

Bokeh contains a lot more tools than the ones listed above.

You can find more examples
in [Plot tools](https://docs.bokeh.org/en/latest/docs/user_guide/interaction/tools.html)
in the Bokeh user guide. A full list of tools is available in the entry for
[tools](https://docs.bokeh.org/en/latest/docs/reference/models/tools.html) in the
Bokeh reference guide.

### Defining active tools

You can **change which tools are active** by setting the ``active_drag``, ``active_inspect``,
``active_scroll``, and ``active_tap`` properties of a toolbar.

The reason why Bokeh has separate properties for different types of tools is that
you can have several tools of different types active at the same time. For example,
you can have an active scroll tool and an active inspect tool at the same time.

The following code cell uses the ``xpan`` tool as its active drag tool.
Update the code in line 7 to make the ``ypan`` tool active by default instead:

In [ ]:
from bokeh.plotting import figure

# set up a figure with a toolbar
p = figure(
    height=300,
    tools=("reset", "pan", "ypan", "xpan", "save"),
    active_drag="xpan",  # 🔁 update this line to make "ypan" the active drag tool
)

p.line(x=list(range(10)), y=list(range(10)))
show(p)

### HoverTool and tooltips

The `HoverTool` is a special tool that **displays a tooltip when the user hovers the
mouse over a data point or taps on a data point**.

To enable tooltips, you need to add a `hover` tool to the list of tools. You can then
use the `tooltips` argument to specify which data fields to display in the tooltip.

Run the cell below and hover over the data points:

In [ ]:
from bokeh.models import ColumnDataSource

source = ColumnDataSource(
    data={
        "x": [1, 2, 3, 6, 9],
        "y": [3, 7, 8, 2, 1],
    }
)

p = figure(
    toolbar_location=None,
    tools="hover",  # add the hover tool to the figure (the hover tool will work even if the toolbar is hidden)
    tooltips="Data point is at @x, @y",  # define a tooltip using data from the x and y columns
    height=300,
)

# add renderers
p.scatter("x", "y", size=25, source=source)
p.line("x", "y", line_width=2, source=source)

# show the results
show(p)

The text that is displayed in the tooltip uses **special syntax to refer to data from
the ColumnDataSource**.

In the example above, the tooltip text is `"Data point is at @x, @y`. The `@x`
and `@y` are replaced with the actual value of the `x` and `y` columns in the
ColumnDataSource at that point.

#### Tooltip fields

There are several **other field names** that you can use in tooltips. See
[Basic tooltips](https://docs.bokeh.org/en/latest/docs/user_guide/interaction/tools.html#basic-tooltips)
in the user guide for an overview.

#### Tooltip formatting

Similar to how you defined the number format for axes, you can **format the numbers in
the tooltips**. To define the format of a data field in a tooltip string, add a format
string in curly braces after the field name. For example:
* `"@x{$0.00}"` will display the value of the `x` column with two decimal places.
* `"@y{($ 0.00 a)}"` will display the value of the `y` column as a Dollar value with two
  decimal places and use SI prefixes (k, M, G, etc.) for large numbers.
* `"@x{(0,0)}"` will display the value of the `x` column as a number with a thousand
  separator.
See [Formatting tooltip fields](https://docs.bokeh.org/en/latest/docs/user_guide/interaction/tools.html#formatting-tooltip-fields) in the user guide for more examples.

### Tabular tooltips

Tooltips can also have **multiple lines and use multiple data fields**. Use a list of
tuples to specify tooltips with multiple lines.

For example:

```python
hover = HoverTool(
    tooltips=[
        ("index", "$index"),
        ("(x,y)", "($x, $y)"),
    ]
)
```

### HoverTool and tooltip example

With the HoverTool, the ColumnDataSource, and the information about customizing visual
properties, you can recreate the "Top 10 carriers by passengers" plot from the demo
dashboard.

First, take a look at the data this plot is visualizing:

In [ ]:
# load data from demo data set
df = data.get_biggest_airlines_by_passengers()
df.head(10)

This table contains the following columns:
* `unique_carrier_name`: full names of the carriers
* `unique_carrier`: IATA codes of the carriers
* `passengers`: the total number of passengers flown in 2021 for each
of the carriers.
* `position`: the position of the carrier in a ranking based on the number of passengers
  flown in 2021.

The next code cell contains the code to create the plot. After running this cell, you
can hover over the blue bars to see the tooltip for each of the carriers:

In [ ]:
from bokeh.models import NumeralTickFormatter, TapTool

# load data for ColumnDataSource from demo data set
source = ColumnDataSource(data.get_biggest_airlines_by_passengers())

# define tooltips as a list of tuples
TOOLTIPS = [
    ("Position", "@position"),
    ("Carrier", "@unique_carrier_name"),
    ("Passengers", "@passengers{(0,0)}"),
]

# set up a figure with the tooltips to use the TOOLTIPS list
largest_carriers_plot = figure(
    x_range=source.data["unique_carrier_name"][:10],
    title="Top 10 carriers by passengers (domestic routes)",
    height=300,
    tooltips=TOOLTIPS,
)

# create a vbar renderer with the data from the ColumnDataSource
carriers_vbar = largest_carriers_plot.vbar(
    x="unique_carrier_name",
    top="passengers",
    source=source,
    legend_label="Passengers",
    width=0.6,
)

# customize the appearance of the grid and x-axis
largest_carriers_plot.xgrid.grid_line_color = None
largest_carriers_plot.yaxis.formatter = NumeralTickFormatter(format="0,0")
largest_carriers_plot.xaxis.major_label_orientation = 0.8  # rotate labels by roughly pi/4

show(largest_carriers_plot)

## RangeTool

The RangeTool allows users to maintain a macroscopic perspective of their data from a 'minimap' view, while navigating through particular regions of interest in a linked plot. This dual view capability is particularly useful for very large time series, images, maps, or scatter plots, where you often need to zoom in on a specific range while keeping an overview of the entire dataset.

### Using the RangeTool
The RangeTool enables interactive definition of a region in one plot that results in a zoomed view on another plot. The example below uses the RangeTool to help navigate through a simple line plot:

In [ ]:
import numpy as np
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, RangeTool
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()

n_points = 3000
x_values = np.linspace(0, 100, n_points)
y_values = np.random.randn(n_points).cumsum()

source = ColumnDataSource(data=dict(x=x_values, y=y_values))

detailed_plot = figure(
    width=800,
    height=300,
    tools=["xpan", "xzoom_in", "xzoom_out", "reset", "wheel_zoom"],
    toolbar_location="above",
    active_scroll="wheel_zoom",
    background_fill_color="#efefef",
    x_range=(22, 30),
)

detailed_plot.line("x", "y", source=source)

minimap = figure(
    width=detailed_plot.width,
    height=150,
    tools="",
    toolbar_location=None,
    background_fill_color=detailed_plot.background_fill_color,
    title="Drag the middle or edges of the selection box below, or double click to start a new box"
)

minimap.line("x", "y", source=source)
minimap.x_range.range_padding = 0
minimap.ygrid.grid_line_color = None

range_tool = RangeTool(x_range=detailed_plot.x_range, y_range=detailed_plot.y_range, start_gesture='tap')
range_tool.overlay.fill_color = "darkblue"
range_tool.overlay.fill_alpha = 0.3
range_tool.overlay.inverted = True
range_tool.overlay.use_handles = True
minimap.add_tools(range_tool)

show(column(detailed_plot, minimap))

Selection of `RangeTool` features shown above:
- `start_gesture`: Set to `'tap'` to allow you to start a new box selection by double tapping.
- `inverted`: Set to `True` to reverse the highlighting, muting the non-selected regions.
- `use_handles`: Set to `True` to add customizable interaction handles for moving (middle) and resizing (edges) the selection box.

## Aside: WebGL

Bokeh draws graphics on web pages using HTML Canvas by default. For larger datasets there is another option which is to use WebGL. This provides hardware acceleration of graphics if you have a processor or graphics cards that supports it.

Let's look at an example that compares Canvas and WebGL side-by-side, using a ``hex_tile`` plot which is 2D histogram using hexagons.

In [ ]:
from bokeh.layouts import row
from bokeh.plotting import figure, show
from bokeh.transform import linear_cmap
from bokeh.util.hex import hexbin
import numpy as np

In [ ]:
n = 5_000_000
x = np.random.standard_normal(n)
y = np.random.standard_normal(n)

bins = hexbin(x, y, 0.01)
print(f"Number of hexagons: {len(bins):,}")

In [ ]:
ps = []
for backend in (["canvas", "webgl"]):
    p = figure(title=backend, tools="pan, wheel_zoom, box_select, reset",
               output_backend=backend, width=450, height=450, lod_threshold=None)
    p.grid.visible = False
    p.hex_tile(q="q", r="r", size=1, line_color=None, source=bins,
               fill_color=linear_cmap('counts', 'Plasma256', 0, max(bins.counts)))
    ps.append(p)
show(row(ps))

Interact with the plots to see how much faster WebGL is in your browser. Things to try:

* Pan using the mouse
* Zoom using the mouse wheel
* Select a region of the plot such as one quarter of it, and repeat the pan and zoom

### How much faster is WebGL?

It is difficult to generalize, it depends on the graphics processing hardware you have available. Try it out to see.

### How do you enable WebGL?

By using the ``output_backend="webgl"`` keyword argument when creating a ``figure``.

### What glyphs are supported by WebGL?

Not all Bokeh glyphs have WebGL support yet, but we are adding more all the time. If you have chosen to use WebGL then Bokeh will use WebGL for the glyphs that it can, and drop back to using Canvas for those not yet supported.

For a full list of glyphs supported using WebGL see the [list of supported glyphs in the Bokeh documentation](https://docs.bokeh.org/en/latest/docs/user_guide/output/webgl.html#supported-glyphs).

# Next section

<a href="09_more_plot_types.ipynb" target="_blank">
    <img src="assets/arrow.svg" alt="Next section" width="100" align="right">
</a>

In the [next chapter](09_more_plot_types.ipynb), you'll learn about more plot types:
Map plots and wedge plots.